In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from scipy import stats
from sklearn.svm import SVR
pd.options.display.float_format = '{: 3f}'.format
significanceLevel = 0.05
testSetSize = 0.25 #percentage of total set, same as paper
k = 10 # for K-fold cross-validation
# for this application we don't really care to much about FP vs FN or are very concerned
# with punishing bad errors as long as the average error is still low
# interpretebility is also key here, so we will use MAE
lossFunction = mean_squared_error
moddels = [('Linear',linear_model.LinearRegression()),
           ('AdaBoost',AdaBoostRegressor()),
           ('Dummy',DummyRegressor()),
           ('Random Forest',RandomForestRegressor()),
          ('Gradient Boost',GradientBoostingRegressor()),
           ('SVM(RBF)',SVR(kernel='rbf', C=1e3, gamma=0.1)),
           ('SVM(Linear)',SVR(kernel='linear', C=1e3)),
           ('SVM(Poly)', SVR(kernel='poly', C=1e3, degree=2))]


''' TRY USING PolynomialFeatures, cross_validate, RandomForestRegressor'''

' TRY USING PolynomialFeatures, cross_validate, RandomForestRegressor'

In [25]:
# Load and clean the data
data = pd.read_csv("../data/energydata_complete.csv").dropna(how='any')
data['date'] = pd.to_datetime(data['date'])
data.set_index('date',inplace = True)
data = data.astype(np.float32)
y = data['Appliances']
X = data.iloc[:,1:]   
data.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60.000000,30.000000,19.889999,47.596668,19.200001,44.790001,19.790001,44.730000,19.000000,45.566666,...,17.033333,45.529999,6.600000,733.500000,92.000000,7.000000,63.000000,5.300000,13.275434,13.275434
2016-01-11 17:10:00,60.000000,30.000000,19.889999,46.693333,19.200001,44.722500,19.790001,44.790001,19.000000,45.992500,...,17.066668,45.560001,6.483333,733.599976,92.000000,6.666667,59.166668,5.200000,18.606195,18.606195
2016-01-11 17:20:00,50.000000,30.000000,19.889999,46.299999,19.200001,44.626667,19.790001,44.933334,18.926666,45.889999,...,17.000000,45.500000,6.366667,733.700012,92.000000,6.333333,55.333332,5.100000,28.642668,28.642668
2016-01-11 17:30:00,50.000000,40.000000,19.889999,46.066666,19.200001,44.590000,19.790001,45.000000,18.889999,45.723331,...,17.000000,45.400002,6.250000,733.799988,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
2016-01-11 17:40:00,60.000000,40.000000,19.889999,46.333332,19.200001,44.529999,19.790001,45.000000,18.889999,45.529999,...,17.000000,45.400002,6.133333,733.900024,92.000000,5.666667,47.666668,4.900000,10.084097,10.084097


In [26]:
# Prepare the data
# taken from http://scikit-learn.org/stable/modules/cross_validation.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSetSize)

In [27]:
# We're looking for strongly correlated features to identify important ones
# the correlation threshold is arbitrarily chosen, but to use this
# we must be certain enough that this is actually the case
# So we also test against the significance level
featureSet = []
for col in data.columns:
    r,p = stats.pearsonr(data['Appliances'],data[col])
    if abs(r) > 0.08 and p < significanceLevel and col!= 'Appliances':
        print("{:<13}({: 5f},{: 5f})".format(col,r,p))
        featureSet.append(col)

lights       ( 0.197278, 0.000000)
RH_1         ( 0.086031, 0.000000)
T2           ( 0.120073, 0.000000)
T3           ( 0.085060, 0.000000)
T6           ( 0.117638, 0.000000)
RH_6         (-0.083178, 0.000000)
RH_8         (-0.094039, 0.000000)
T_out        ( 0.099155, 0.000000)
RH_out       (-0.152282, 0.000000)
Windspeed    ( 0.087122, 0.000000)


In [28]:
# model.fit(X_train,y=y_train)
# mean_squared_error(y_test,model.predict(X_test))
maxNameLength = str(max([len(name) for name,model in moddels] + [len("Full Feature set"),len("Reduced feature set")]))
print("{:<{width}}{:>{width}}  {:>{width}}".format("","Full Feature set,","Reduced feature set",width = maxNameLength))

for name,model in moddels:
    fullSetResults = cross_val_score(model,X_train,y=y_train,cv=k,scoring='neg_mean_absolute_error',n_jobs=-1)
    redSetResults = cross_val_score(model,X_train[featureSet],y=y_train,cv=k,scoring='neg_mean_absolute_error',n_jobs=-1)
    print("{:<{width}{name}}{: <{width}.{prec}f{fullAcc},{}} {: <{width}.{prec}f}".format(name,
                                       -1*.mean(),
                                      -1*.mean(),
                                                              width = str(maxNameLength),
                                                                     prec='3'))


                     Full Feature set,  Reduced feature set
Linear              52.621              54.363             
AdaBoost            153.168             180.575            
Dummy               60.358              60.358             
Random Forest       38.141              39.643             
Gradient Boost      47.596              50.095             


In [ ]:
'''
64:
                     Full Feature set,  Reduced feature set
Linear              53.448              55.130             
AdaBoost            135.047             132.924            
Dummy               60.966              60.966             
Random Forest       38.741              39.924             
Gradient Boost      48.235              50.568 

32:
                     Full Feature set,  Reduced feature set
Linear              52.621              54.363             
AdaBoost            153.168             180.575    
Dummy               60.358              60.358             
Random Forest       38.141              39.643             
Gradient Boost      47.596              50.095 

'''

In [33]:
fullSet = cross_validate(model,X_train,y=y_train,cv=k,scoring='neg_mean_absolute_error',n_jobs=-1)
print(fullSet['test_score'].mean(),fullSet['fit_time'].mean())

-47.595880028805034 1.8134677410125732
